In [6]:
#!pip install openpyxl
!pip install PyPDF2

import openpyxl
import PyPDF2
import pandas as pd
import os
import re

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.3 MB/s eta 0:00:00


# Importation des bases # 


In [50]:
# Obtenez le chemin du répertoire parent du répertoire src
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Construisez le chemin complet vers votre fichier Excel
fichier_excel = os.path.join(base_dir, 'data', 'text', 'Base de donées anti trans.xlsx')

# Chargez le fichier Excel dans un DataFrame
man2 = pd.read_excel(fichier_excel, sheet_name=2)
long = pd.read_excel(fichier_excel, sheet_name=3)

## Gestion de la base de données avec les longs PDF ##

In [51]:
long

,ID ASSO,URL,Type de document,ID ARTICLE,Date,contenu
0,11,https://soseducation.org/docs/notes-etudes-ent...,Note de synthèse,NaN,NaN,NaN
1,11,https://soseducation.org/docs/notes-etudes-ent...,Note de synthèse,NaN,NaN,NaN
2,23,https://www.observatoirepetitesirene.org/post/...,NaN,NaN,NaN,NaN
3,22,https://www.observatoirepetitesirene.org/_file...,Rapport médical,NaN,NaN,NaN


In [52]:
# Obtenez le chemin du répertoire parent du répertoire src
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Construisez le chemin complet vers votre fichier pdf
fichier_pdf1 = os.path.join(base_dir, 'data', 'pdf', 'SOSEDUC1.pdf')
fichier_pdf2 = os.path.join(base_dir, 'data', 'pdf', 'SOSEDUC2.pdf')
fichier_pdf3 = os.path.join(base_dir, 'data', 'pdf', 'OPSReport.pdf')
fichier_pdf4 = os.path.join(base_dir, 'data', 'pdf', 'OPS2.pdf')

def lire_contenu_pdf(chemin_pdf):
    with open(chemin_pdf, 'rb') as fichier:
        lecteur_pdf = PyPDF2.PdfReader(fichier)
        contenu_textuel = ''
        for page_num in range(len(lecteur_pdf.pages)):
            page = lecteur_pdf.pages[page_num]
            contenu_textuel += page.extract_text()
    return contenu_textuel

sosedu1 = lire_contenu_pdf(fichier_pdf1)
#sosedu2 = lire_contenu_pdf(fichier_pdf2) (ça marche pas et honnêtement ça parle d'éducation sexuelle en général donc on peut probablement s'en passer 
ops_rep = lire_contenu_pdf(fichier_pdf3)
ops2 = lire_contenu_pdf(fichier_pdf4)

In [53]:

# Nettoyage de sosedu1
texte_initial = str(sosedu1)

# Mettre tout le texte en minuscule
texte_en_minuscule = texte_initial.lower()

# Supprimer les caractères indésirables (\n, \x00, etc.)
texte_nettoye = re.sub(r'\\n|\\x[0-9A-Fa-f]+', ' ', texte_en_minuscule)

# Supprimer les mentions spécifiques (SOS Éducation, REMERCIEMENTS, ANNEXE)
mentions_a_supprimer = ['sos éducation', 'remerciements', 'annexe', '17 décembre 2021', '\n', '\x00', ' p ']

for mention in mentions_a_supprimer:
    texte_nettoye = texte_nettoye.replace(mention, '')

# Supprimer les liens hypertextes
texte_nettoye = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', texte_nettoye)

# Supprimer les chaînes de caractères contenant des "@"
texte_nettoye = re.sub(r'@', '', texte_nettoye)

# Supprimer les espaces multiples
texte_nettoye = re.sub(r'\s+', ' ', texte_nettoye).strip()

texte_nettoye
sosedu1 = texte_nettoye

Notons que le nettoyage effectué précédemment est très imparfait et qu'on a plusieurs espaces manquant, mais cela devrait être peut être négligeable dans le corpus final

In [54]:
texte_initial = str(ops_rep)
# Mettre tout le texte en minuscule
texte_en_minuscule = texte_initial.lower()

# Supprimer les caractères indésirables (\n, \x00, etc.)
texte_nettoye = re.sub(r'\\n|\\x[0-9A-Fa-f]+', ' ', texte_en_minuscule)

# Supprimer les mentions spécifiques (SOS Éducation, REMERCIEMENTS, ANNEXE)
mentions_a_supprimer = ['.', 'ibid', '\n' ]

for mention in mentions_a_supprimer:
    texte_nettoye = texte_nettoye.replace(mention, '')
# Supprimer les liens hypertextes
texte_nettoye = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', texte_nettoye)

texte_nettoye
ops_rep = texte_nettoye 

Précedemment, le nettoyage est plus propre que le précédent mais il reste encore beaucoup de références qu'on aperçoit en bas de page dans le texte

In [55]:
# Texte initial (ops2)
texte_initial = str(ops2)

# Mettre tout le texte en minuscule
texte_en_minuscule = texte_initial.lower()

# Supprimer les caractères indésirables (\n, \x00, etc.)
texte_nettoye = re.sub(r'\\n|\\x[0-9A-Fa-f]+', ' ', texte_en_minuscule)

# Supprimer les mentions spécifiques (SOS Éducation, REMERCIEMENTS, ANNEXE)
mentions_a_supprimer = ['.', 'ibid', "\\",'\n']

for mention in mentions_a_supprimer:
    texte_nettoye = texte_nettoye.replace(mention, '')

# Supprimer les liens hypertextes
texte_nettoye = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', texte_nettoye)

# Supprimer tout le texte après la mention "Ressources supplémentaires"
texte_nettoye = re.sub(r'ressources supplémentaires.*', '', texte_nettoye)

# Supprimer les espaces multiples
texte_nettoye = re.sub(r'\s+', ' ', texte_nettoye).strip()

texte_nettoye
ops2 = texte_nettoye

Nettoyage imparfait, encore une fois. 

### Création de la base long en prenant les textes nettoyés dans contenu ###

In [56]:
long = long.drop(1)
long

,ID ASSO,URL,Type de document,ID ARTICLE,Date,contenu
0,11,https://soseducation.org/docs/notes-etudes-ent...,Note de synthèse,NaN,NaN,NaN
2,23,https://www.observatoirepetitesirene.org/post/...,NaN,NaN,NaN,NaN
3,22,https://www.observatoirepetitesirene.org/_file...,Rapport médical,NaN,NaN,NaN


In [57]:
long.loc[0, 'contenu'] = sosedu1

/tmp/ipykernel_1596/2253050767.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'la question tr ansg enrechez les enfantscessez de dégenrer puis regenrer les enfants. laissez-les rêver, laissez-les grandir, laissez-les vivre !synthèse documentée pour comprendre les dang ersde l’activisme politique transa rma tif sur les jeunes« si vous vous demandez si vous êtes trans,c’est que vous l’êtes probablement. » | la question transgenre chez les enfants« si je m’adressais à une fille atteinte de dysphorie de genrequi déteste son corps autant que j’ai détesté le mien,je lui dirais de sortir dehors, d’aller jouer dans la boue,de grimper aux arbres, de trouver une manièred’habiter son corps selon ses propres volontés. »« le pire moment, c’est quand j’ai réalisé qu’en réalité,j’avais l’apparence d’une fille tout à fait normale,que j’étais mince et jolie. maintenant à cause de ma transition,j’aurai pour toujours u

In [58]:
long.loc[2, 'contenu'] = ops_rep

In [59]:
long.loc[3, 'contenu'] = ops2

In [60]:
long

,ID ASSO,URL,Type de document,ID ARTICLE,Date,contenu
0,11,https://soseducation.org/docs/notes-etudes-ent...,Note de synthèse,NaN,NaN,la question tr ansg enrechez les enfantscessez...
2,23,https://www.observatoirepetitesirene.org/post/...,NaN,NaN,NaN,1 les traitements hormonaux utilisés pour les...
3,22,https://www.observatoirepetitesirene.org/_file...,Rapport médical,NaN,NaN,11 filles autistes et identité de genre 8 12 f...


## Merge avec la base manuelle 2 ##

In [63]:
df = pd.concat([long, man2], ignore_index=True)
df

,ID ASSO,URL,Type de document,ID ARTICLE,Date,contenu
0,11,https://soseducation.org/docs/notes-etudes-ent...,Note de synthèse,NaN,NaN,la question tr ansg enrechez les enfantscessez...
1,23,https://www.observatoirepetitesirene.org/post/...,NaN,NaN,NaN,1 les traitements hormonaux utilisés pour les...
2,22,https://www.observatoirepetitesirene.org/_file...,Rapport médical,NaN,NaN,11 filles autistes et identité de genre 8 12 f...
3,11,https://soseducation.org/petitions-mobilisatio...,Pétition,NaN,NaN,PÉTITION À l’attention de monsieur Gabriel Att...
4,11,https://soseducation.org/docs/mobilisations/al...,Lettre,NaN,NaN,Monsieur le Ministre de l’Éducation nationale ...
...,...,...,...,...,...,...
149,5,https://www.egaliteetreconciliation.fr/Feminis...,Article d'opinion,NaN,2022-03-10 00:00:00,Le mois dernier nous publiions la première par...
150,5,https://www.egaliteetreconciliation.fr/Feminis...,Article d'opinion,NaN,2022-02-07 00:00:00,Alors que l’affaire Brigitte Macron/Jean-Miche...
151,5,https://www.egaliteetreconciliation.fr/La-tran...,Article d'opinion,NaN,2020-02-02 00:00:00,"Il a fait ce qu’on appelle une transition, il ..."
152,5,https://www.egaliteetreconciliation.fr/Le-femi...,Article d'opinion,NaN,2019-11-03 00:00:00,Le titre est un peu alambiqué mais on va dével...


In [ ]:
#On exporte 
# Obtenez le chemin du répertoire parent du répertoire src
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Construisez le chemin complet vers le répertoire de données
chemin_data = os.path.join(base_dir, 'data')

# Enregistrez le DataFrame dans un fichier CSV dans le répertoire data
df.to_csv(os.path.join(chemin_data, 'base_def.tsv'), sep='\t', index=False)